In [40]:
# Testing the relationship of token price vs TVL for protocols with token emissions
# i.e. does token price correlate with liquidity for DEXs with token emissions models?
import sys
sys.path.append("..")
import defillama_utils as dfl
import coingecko_utils as cg
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

protocol = 'velodrome'
token_address = '0x3c8b650257cfb5f272f799f5e2b4e65093a11a05'
chain = 'optimism'

In [41]:
dfl_df = dfl.get_single_tvl(protocol,[chain])
dfl_df = dfl_df[dfl_df['date'] < pd.to_datetime("today").floor('d')]
dfl_df['date'] = dfl_df['date'].dt.date

# display(dfl_df)

In [42]:
prices_df = dfl.get_historical_defillama_prices([token_address], chain)
prices_df['chain'] = prices_df['chain'].str.capitalize()
prices_df['date'] = pd.to_datetime(prices_df['date']).dt.date

# display(prices_df)

In [43]:
combo_df = dfl_df.merge(prices_df,on=['date','chain'],how='inner')
display(combo_df)

,date,token,token_value,usd_value,total_app_tvl,protocol,chain,category,token_address,symbol,decimals,timestamp,price
0,2022-06-03,totalLiquidityUSD,3.045954e+07,30459541.99734,3.045954e+07,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1654276923,0.047720
1,2022-06-04,totalLiquidityUSD,3.919414e+07,39194135.18038,3.919414e+07,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1654363192,0.039807
2,2022-06-05,totalLiquidityUSD,3.929542e+07,39295424.83857,3.929542e+07,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1654449520,0.036047
3,2022-06-06,totalLiquidityUSD,4.203704e+07,42037041.17403,4.203704e+07,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1654535937,0.036564
4,2022-06-07,totalLiquidityUSD,4.871631e+07,48716309.76363,4.871631e+07,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1654622397,0.026990
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,2023-02-23,totalLiquidityUSD,2.714084e+08,271408391.17552,2.714084e+08,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1677172968,0.214464
259,2023-02-24,totalLiquidityUSD,2.809959e+08,280995873.0114,2.809959e+08,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1677259226,0.200090
260,2023-02-25,totalLiquidityUSD,2.947014e+08,294701437.36058,2.947014e+08,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1677345782,0.177420
261,2023-02-26,totalLiquidityUSD,2.962588e+08,296258786.62152,2.962588e+08,velodrome,Optimism,Dexes,0x3c8b650257cfb5f272f799f5e2b4e65093a11a05,VELO,18,1677431990,0.218467


In [44]:
# fig = px.line(combo_df, x='date', y=['total_app_tvl', 'price'], 
#               title='Total Liquidity and Price Over Time')
# # set the labels for each y-axis
# fig.update_yaxes(title_text='Total Liquidity', secondary_y=False)
# fig.update_yaxes(title_text='Price', secondary_y=True)

# fig.show()


combo_df = combo_df[ combo_df['date'] > pd.to_datetime('2022-07-13')]
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=combo_df['date'], y=combo_df['total_app_tvl'], name="TVL"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=combo_df['date'], y=combo_df['price'], name="Token Price"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Total Liquidity and Price Over Time - " + protocol.capitalize()
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="TVL", secondary_y=False)
fig.update_yaxes(title_text="Token Price", secondary_y=True)

fig.show()

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_41697/2802834832.py:10: FutureWarning:

Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.

